In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Reading Data set

In [ ]:
train=pd.read_csv("/kaggle/input/quora-insincere-questions-classification/train.csv")
train.head()

In [ ]:
train.count()

In [ ]:
test=pd.read_csv("/kaggle/input/quora-insincere-questions-classification/test.csv")
test.head()

In [ ]:
test.count()

# Word Cloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 

#            Word Cloud for whole training data set.

In [ ]:
text1 = " ".join(review for review in train.question_text)
wordcloud = WordCloud(background_color="white").generate(text1)

In [ ]:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

Here,
   Will,best,India,make,use,one,think,good are repeated most of the time.

# Word cloud for Negative Response test data set

In [ ]:
Negative_response = " ".join(review for review in train[train["target"]==0].question_text)
wordcloud0 = WordCloud(background_color="white", 
                          mode="RGBA", max_words=1000,).generate(Negative_response)

In [ ]:
plt.imshow(wordcloud0, interpolation='bilinear')
plt.axis("off")
plt.show()

Here is surprise for me....
   For Negative response some word like best,good,use are repeated most of the time,which prety good.

# Word cloud for Positive Response of traing data set

In [ ]:
Positive_response = " ".join(review for review in train[train["target"]==1].question_text)
wordcloud1 = WordCloud(background_color="red", 
                          mode="RGBA", max_words=1000,).generate(Positive_response)

In [ ]:
plt.imshow(wordcloud1, interpolation='bilinear')
plt.axis("off")
plt.show()

# **Word Frequency plot of sincere**

In [ ]:
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text import FreqDistVisualizer

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
docs       = vectorizer.fit_transform(text for text in train['question_text'])
features   = vectorizer.get_feature_names()

visualizer = FreqDistVisualizer(
    features=features#, size=(1080, 720)
)
visualizer.fit(docs)
visualizer.show()

# It's very big data and I can't run any algorithm due to memory size but simple coding available for beginner with statistical background.

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 

stops = stopwords.words('english')

corpus = []
for i in range(0, 1306122):
    question_text = re.sub('[^a-zA-Z]', ' ', train['question_text'][i])
    question_text = question_text.lower()
    question_text = question_text.split()
    ps = PorterStemmer()
    question_text = [ps.stem(word) for word in question_text if not word in set(stops)]
    question_text = ' '.join(question_text)
    corpus.append(question_text)

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = train.iloc[:,train.columns=="target"]

from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3, 
                                                    random_state=1,
                                                    stratify=y)

from xgboost import XGBClassifier
clf=XGBClassifier(random_state=2020,n_jobs=-1)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
from sklearn import metrics
print("Accuracy of XGboost Algorithm model is:",
metrics.accuracy_score(y_test, y_pred)*100)
